In [1]:
import pandas as pd
import os

In [2]:
TEXT_EMBEDDING_PATH = "../data/artifacts/google_bert_base_multilingual_cased/google_bert_base_multilingual_cased/bert_base_multilingual_cased.parquet"

In [3]:
SOURCE = "ebnerd_demo"
SPLIT = 'validation'

READ_TIME_MIN = 7
READ_TIME_MAX = 600

In [4]:
DATA_PATH = f"../data/{SOURCE}/{SPLIT}/"

In [5]:
df_behave = pd.read_parquet(f"{DATA_PATH}/behaviors.parquet")

In [6]:
df_behave

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,144772,NaN,2023-05-30 14:21:34,29.0,NaN,2,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],76658,False,NaN,NaN,NaN,False,29,7.0,59.0
1,144777,NaN,2023-05-30 14:22:11,10.0,NaN,2,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],76658,False,NaN,NaN,NaN,False,29,58.0,98.0
2,196487,NaN,2023-05-27 19:54:18,16.0,NaN,2,"[9279095, 9784273, 9784275, 9784506, 9784444, ...",[9782806],760446,False,NaN,NaN,NaN,False,220,64.0,100.0
3,196495,NaN,2023-05-27 19:53:48,25.0,NaN,2,"[9784575, 9784607, 9784559, 9784662, 9783852, ...",[9782656],760446,False,NaN,NaN,NaN,False,220,4.0,56.0
4,196496,NaN,2023-05-27 19:56:28,11.0,NaN,2,"[9784137, 9784298, 9779370, 9782517, 9777324, ...",[9777324],760446,False,NaN,NaN,NaN,False,220,65.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25351,578776841,NaN,2023-05-26 20:46:25,63.0,100.0,2,"[9754112, 9781257, 9782234, 9695420, 9780467, ...",[9783317],1167944,False,NaN,NaN,NaN,False,105113,23.0,100.0
25352,578776846,9783317.0,2023-05-26 20:47:28,23.0,100.0,2,"[9783237, 9781257, 9783317, 9783276, 9754112]",[9754112],1167944,False,NaN,NaN,NaN,False,105113,87.0,87.0
25353,578777590,9780467.0,2023-05-26 20:43:52,21.0,100.0,2,"[9783334, 9780467, 9742735, 9775915, 9782517]",[9782517],1171223,False,NaN,NaN,NaN,False,113995,64.0,70.0
25354,578783544,NaN,2023-05-26 18:59:17,10.0,NaN,2,"[9783164, 9780517, 9783291, 9783004, 9782998]",[9783004],1358150,False,NaN,NaN,NaN,False,183279,8.0,61.0


In [7]:
df_behave = df_behave[["user_id", "impression_time", "article_ids_clicked", "next_read_time"]]

In [8]:
df_behave["article_ids_clicked"].apply(lambda x: len(x)).value_counts()

1    25227
2      119
3        5
4        3
7        1
6        1
Name: article_ids_clicked, dtype: int64

In [9]:
# as the most clicked article is 1, we can take only the first article
df_behave["article_id"] = df_behave["article_ids_clicked"].apply(lambda x: x[0])
df_behave = df_behave.drop(columns=["article_ids_clicked"])

/var/folders/yb/pvj_hqps2976hrvlzg0k58ym0000gq/T/ipykernel_63576/2801589552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_behave["article_id"] = df_behave["article_ids_clicked"].apply(lambda x: x[0])


In [10]:
df_behave.columns = ["user_id", "impression_ts", "article_read_time", "article_id"]

In [11]:
# expanding user's history
df_history = pd.read_parquet(f"{DATA_PATH}/history.parquet")

In [12]:
df_history = df_history.explode(['impression_time_fixed', 'scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed'], ignore_index=True)
df_history = df_history[["user_id", "impression_time_fixed", "article_id_fixed", "read_time_fixed"]]
df_history.columns = ["user_id", "impression_ts", "article_id", "article_read_time"]
df_history

,user_id,impression_ts,article_id,article_read_time
0,750497,2023-05-04 09:42:39,9749224,49.0
1,750497,2023-05-04 09:43:28,9749156,5.0
2,750497,2023-05-04 09:43:34,9749224,7.0
3,750497,2023-05-04 09:43:41,9748948,151.0
4,750497,2023-05-04 09:46:13,9748980,214.0
...,...,...,...,...
240053,2385386,2023-05-12 09:18:31,9756886,72.0
240054,2385386,2023-05-12 09:19:44,9761363,19.0
240055,2385386,2023-05-12 09:20:03,9761469,4.0
240056,2385386,2023-05-12 09:20:08,9761363,3.0


In [13]:
# merging history with the main dataframe
df_behave = pd.concat([df_behave, df_history], ignore_index=True)
df_behave = df_behave.sort_values(by=["user_id", "impression_ts"])
df_behave

,user_id,impression_ts,article_read_time,article_id
28556,19181,2023-05-04 08:03:50,48.0,9749036
28557,19181,2023-05-04 08:40:52,2.0,9749092
28558,19181,2023-05-04 08:40:58,2.0,9749156
28559,19181,2023-05-04 08:41:07,10.0,9748916
28560,19181,2023-05-04 08:41:28,6.0,9747959
...,...,...,...,...
9539,2585449,2023-05-31 10:13:13,216.0,9771579
1558,2585449,2023-05-31 13:55:13,77.0,9789417
1560,2585449,2023-05-31 13:55:22,9.0,9673564
1559,2585449,2023-05-31 13:56:50,62.0,9787846


In [14]:
# doing some data cleaning

count_before = df_behave.shape[0]

# removing records with low read time
df_behave = df_behave.loc[df_behave["article_read_time"] >= READ_TIME_MIN]
df_behave = df_behave.loc[df_behave["article_read_time"] <= READ_TIME_MAX]

print(f"Removed {count_before - df_behave.shape[0]} records, which is {100 * (count_before - df_behave.shape[0]) / count_before}% of the data")

Removed 81217 records, which is 30.60011905928097% of the data


In [15]:
articles = pd.read_parquet(f"../data/{SOURCE}/articles.parquet")

In [16]:
articles

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,...,[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,"[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11772,9803492,Vilde billeder: Vulkan i udbrud i ferieparadis,Der er gang i vulkanen på Hawaiis største ø,2023-06-29 06:49:26,False,Det spyer med lava fra vulkanen Kilauea på Haw...,2023-06-08 05:49:20,"[9803493, 9803494, 9803495, 9803495, 9803494]",article_default,https://ekstrabladet.dk/nyheder/samfund/vilde-...,...,"[LOC, LOC, PER, ORG, ORG]","[Katastrofe, Vejr, Større katastrofe]",118,[133],nyheder,535989.0,100120.0,4112624.0,0.6095,Neutral
11773,9803505,Flyvende Antonsen knuser topspiller,"Verdens nummer syv, Chou Tien-Chen, fik ikke e...",2023-06-29 06:49:26,False,Anders Antonsen har holdt pause fra badmintonb...,2023-06-08 05:54:06,[9803516],article_default,https://ekstrabladet.dk/sport/anden_sport/badm...,...,"[PER, PROD, PER, PER, MISC, MISC, PER, PER, LO...","[Kendt, Begivenhed, Sport, Ketcher- og batspor...",142,"[327, 330]",sport,13320.0,959.0,55691.0,0.8884,Positive
11774,9803525,"Dansk skuespiller: - Jeg nægtede, at jeg var syg",Julie R. Ølgaard fik akut kejsersnit og fødte ...,2023-06-29 06:49:26,False,"Mens hun lå søvnløs, lød kakofonien fra baggår...",2023-06-08 06:45:46,"[9803518, 9803519, 9803520, 9803521, 9803522, ...",article_default,https://ekstrabladet.dk/underholdning/dkkendte...,...,"[PER, PROD, PER, PER, PER, PER, MISC]","[Kendt, Livsstil, Familieliv, Underholdning, F...",414,[425],underholdning,315391.0,50361.0,2550671.0,0.7737,Negative
11775,9803560,Så slemt er det: 14.000 huse er oversvømmet,Tusindvis af huse står under vand i Kherson-re...,2023-06-29 06:49:26,False,Et område på omkring 600 kvadratkilometer står...,2023-06-08 06:25:42,None,article_default,https://ekstrabladet.dk/nyheder/saa-slemt-er-d...,...,"[LOC, LOC, LOC, PROD, PER, LOC, ORG, ORG, LOC]","[International politik, Katastrofe, Større kat...",118,[],nyheder,21318.0,1237.0,67514.0,0

In [17]:
# fill total_read_time with median, where it is missing
articles = articles.fillna(articles["total_read_time"].median())
articles["article_ctr"] = articles["total_pageviews"] / articles["total_inviews"]
# fill ctr with median, where it is missing
articles = articles.fillna(articles["article_ctr"].median())

In [18]:
articles = articles[["article_id", "premium", "article_type", "topics", "category_str", "total_read_time", "sentiment_score", "article_ctr"]]

In [19]:
articles.columns = ["article_id", "article_is_premium", "article_type", "article_topics", "article_category", "article_total_read_time", "article_sentiment", "article_ctr"]

In [20]:
# merging articles with the main dataframe
df_behave = df_behave.merge(articles, on="article_id", how="left")

In [21]:
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr
0,19181,2023-05-04 08:03:50,48.0,9749036,False,article_default,"[Erhverv, Privat virksomhed, Økonomi]",nyheder,7003349.0,0.5285,0.249884
1,19181,2023-05-04 08:41:07,10.0,9748916,False,article_default,"[Begivenhed, Sport, Sundhed, Sygdom og behandl...",sport,6051867.0,0.9135,0.159207
2,19181,2023-05-04 09:53:28,14.0,9749275,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,9433228.0,0.9760,0.270433
3,19181,2023-05-04 13:46:40,547.0,9347884,False,article_default,"[Kendt, Livsstil, Underholdning, Reality]",underholdning,2976663.5,0.8563,1.000000
4,19181,2023-05-04 18:52:58,9.0,9749668,True,article_default,"[Kriminalitet, Erhverv, Kendt, Samfund, Værdie...",nyheder,9550603.0,0.9900,0.121067
...,...,...,...,...,...,...,...,...,...,...,...
184192,2585449,2023-05-31 10:13:13,216.0,9771579,False,article_default,"[Kendt, Begivenhed, Sport, Sundhed, Sygdom og ...",sport,1227700.0,0.9285,0.119506
184193,2585449,2023-05-31 13:55:13,77.0,9789417,False,article_default,"[Kendt, Livsstil, Familieliv, Underholdning, F...",underholdning,5148529.0,0.9210,0.168332
184194,2585449,2023-05-31 13:55:22,9.0,9673564,True,article_default,"[Økonomi, Mikro]",forbrug,5696063.0,0.7402,0.040712
184195,2585449,2023-05-31 13:56:50,62.0,9787846,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,2336959.0,0.9938,0.161884


In [22]:
# loading text embeddings
emb_df = pd.read_parquet(TEXT_EMBEDDING_PATH)
emb_df.columns = ["article_id", "article_emb"]

In [23]:
emb_df

,article_id,article_emb
0,3000022,"[-0.35060593, 0.0034366532, -0.1756858, -0.088..."
1,3000063,"[-0.0034481985, 0.22765873, -0.19700234, 0.060..."
2,3000613,"[-0.03811903, -0.030120859, -0.6928362, 0.0801..."
3,3000700,"[-0.29374197, -0.07282147, -0.0926456, -0.0540..."
4,3000840,"[0.0019190352, -0.010633812, -0.49158585, 0.17..."
...,...,...
125536,9803505,"[-0.05083758, 0.068563275, -0.2283888, 0.09498..."
125537,9803510,"[-0.055143625, 0.15021142, -0.15252301, 0.1173..."
125538,9803525,"[-0.035008177, 0.12463321, -0.20443131, 0.1012..."
125539,9803560,"[-0.15544586, 0.07193958, -0.36763963, 0.08102..."


In [24]:
# merging text embeddings with the main dataframe
df_behave = df_behave.merge(emb_df, on="article_id", how="left")
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr,article_emb
0,19181,2023-05-04 08:03:50,48.0,9749036,False,article_default,"[Erhverv, Privat virksomhed, Økonomi]",nyheder,7003349.0,0.5285,0.249884,"[-0.14355686, 0.07587211, -0.1749185, 0.054935..."
1,19181,2023-05-04 08:41:07,10.0,9748916,False,article_default,"[Begivenhed, Sport, Sundhed, Sygdom og behandl...",sport,6051867.0,0.9135,0.159207,"[-0.06336258, 0.11513184, -0.17666768, 0.15081..."
2,19181,2023-05-04 09:53:28,14.0,9749275,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,9433228.0,0.9760,0.270433,"[-0.0490314, 0.100983106, -0.19836023, 0.16166..."
3,19181,2023-05-04 13:46:40,547.0,9347884,False,article_default,"[Kendt, Livsstil, Underholdning, Reality]",underholdning,2976663.5,0.8563,1.000000,"[-0.074573174, 0.15921499, -0.19931641, 0.1562..."
4,19181,2023-05-04 18:52:58,9.0,9749668,True,article_default,"[Kriminalitet, Erhverv, Kendt, Samfund, Værdie...",nyheder,9550603.0,0.9900,0.121067,"[-0.17662802, 0.14803919, -0.2546618, 0.094485..."
...,...,...,...,...,...,...,...,...,...,...,...,...
184192,2585449,2023-05-31 10:13:13,216.0,9771579,False,article_default,"[Kendt, Begivenhed, Sport, Sundhed, Sygdom og ...",sport,1227700.0,0.9285,0.119506,"[-0.039634164, 0.11201517, 0.020189472, 0.0090..."
184193,2585449,2023-05-31 13:55:13,77.0,9789417,False,article_default,"[Kendt, Livsstil, Familieliv, Underholdning, F...",underholdning,5148529.0,0.9210,0.168332,"[-0.070519224, 0.09460071, -0.22725518, 0.0866..."
184194,2585449,2023-05-31 13:55:22,9.0,9673564,True,article_default,"[Økonomi, Mikro]",forbrug,5696063.0,0.7402,0.040712,"[0.062252264, 0.035441652, -0.0682976, 0.02613..."
184195,2585449,2023-05-31 13:56:50,62.0,9787846,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,2336959.0,0.9938,0.161884,"[-0.016353942, 0.08321255, -0.13511518, 0.1513..."


In [25]:
# writing the final dataframe
os.makedirs(f"../data/{SOURCE}_modified/", exist_ok=True)

df_behave.to_parquet(f"../data/{SOURCE}_modified/{SPLIT}.parquet", index=False)

In [26]:
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr,article_emb
0,19181,2023-05-04 08:03:50,48.0,9749036,False,article_default,"[Erhverv, Privat virksomhed, Økonomi]",nyheder,7003349.0,0.5285,0.249884,"[-0.14355686, 0.07587211, -0.1749185, 0.054935..."
1,19181,2023-05-04 08:41:07,10.0,9748916,False,article_default,"[Begivenhed, Sport, Sundhed, Sygdom og behandl...",sport,6051867.0,0.9135,0.159207,"[-0.06336258, 0.11513184, -0.17666768, 0.15081..."
2,19181,2023-05-04 09:53:28,14.0,9749275,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,9433228.0,0.9760,0.270433,"[-0.0490314, 0.100983106, -0.19836023, 0.16166..."
3,19181,2023-05-04 13:46:40,547.0,9347884,False,article_default,"[Kendt, Livsstil, Underholdning, Reality]",underholdning,2976663.5,0.8563,1.000000,"[-0.074573174, 0.15921499, -0.19931641, 0.1562..."
4,19181,2023-05-04 18:52:58,9.0,9749668,True,article_default,"[Kriminalitet, Erhverv, Kendt, Samfund, Værdie...",nyheder,9550603.0,0.9900,0.121067,"[-0.17662802, 0.14803919, -0.2546618, 0.094485..."
...,...,...,...,...,...,...,...,...,...,...,...,...
184192,2585449,2023-05-31 10:13:13,216.0,9771579,False,article_default,"[Kendt, Begivenhed, Sport, Sundhed, Sygdom og ...",sport,1227700.0,0.9285,0.119506,"[-0.039634164, 0.11201517, 0.020189472, 0.0090..."
184193,2585449,2023-05-31 13:55:13,77.0,9789417,False,article_default,"[Kendt, Livsstil, Familieliv, Underholdning, F...",underholdning,5148529.0,0.9210,0.168332,"[-0.070519224, 0.09460071, -0.22725518, 0.0866..."
184194,2585449,2023-05-31 13:55:22,9.0,9673564,True,article_default,"[Økonomi, Mikro]",forbrug,5696063.0,0.7402,0.040712,"[0.062252264, 0.035441652, -0.0682976, 0.02613..."
184195,2585449,2023-05-31 13:56:50,62.0,9787846,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,2336959.0,0.9938,0.161884,"[-0.016353942, 0.08321255, -0.13511518, 0.1513..."


In [28]:
# additionally join both train and validation data
df_behave_train = pd.read_parquet(f"../data/{SOURCE}_modified/train.parquet")
df_behave_val = pd.read_parquet(f"../data/{SOURCE}_modified/validation.parquet")

df_behave = pd.concat([df_behave_train, df_behave_val], ignore_index=True)
df_behave.to_parquet(f"../data/{SOURCE}_modified/all.parquet", index=False)